<a href="https://colab.research.google.com/github/zrosen3/Colombian-peso-counterfeit-detection/blob/main/Code/hybrid_cnns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load libraries

In [116]:
#Set up google colab working environment
%cd ~/../content
%rm -rf Colombian-peso-counterfeit-detection
!git clone https://github.com/zrosen3/Colombian-peso-counterfeit-detection.git 
%cd /content/Colombian-peso-counterfeit-detection/Code       

/content
Cloning into 'Colombian-peso-counterfeit-detection'...
remote: Enumerating objects: 42262, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 42262 (delta 61), reused 25 (delta 18), pack-reused 42174
Receiving objects: 100% (42262/42262), 883.49 MiB | 29.15 MiB/s, done.
Resolving deltas: 100% (1220/1220), done.
Updating files: 100% (103997/103997), done.
/content/Colombian-peso-counterfeit-detection/Code


In [117]:
import matplotlib.pyplot as plt #For plotting our visualizations
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import cv2
from PIL import Image
import seaborn as sns
import common_functions as cf
import xgboost as xgb



#Load in data

In [118]:
train_ds, test_ds, validation_ds, class_names = cf.LoadData(path = "../Data/Augmented/")


Found 24958 files belonging to 13 classes.
Found 8320 files belonging to 13 classes.
Found 8320 files belonging to 13 classes.


## Create ensemble of CNNs

Create ensemble model

In [128]:
def CNNModel(class_names:str,  conv_layers: list = [32], layers: list = [], learning_rate: float = 0.001, dropout: float = 0.5) -> tf.keras.Sequential:
    """
    Simple straight forward CNN model. this is just for simplicity and testing
    atm. I will make it more modular later once I know what we are doing
    Args:
        class_names: list of the classification names
        conv_layers: list of how many filters each convolutional layer should use
        layers: list with the sizes of each hidden layer
        learning_rate: the learning rate for the optimizer
        dropout: the dropout rate for the model
    Returns:
        `tf.keras.Sequential` - a constructed tf model
    """
    tf.keras.backend.clear_session()

    model = tf.keras.Sequential(name = str(i))
    model.add(tf.keras.layers.Rescaling(1./255))
    for filter_count in conv_layers:
        model.add(tf.keras.layers.Conv2D(filter_count, 3, activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D())
    model.add(tf.keras.layers.Flatten())
    for layer_count in layers:
        model.add(tf.keras.layers.Dense(layer_count, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=dropout))
    model.add(tf.keras.layers.Dense(len(class_names), activation = 'softmax'))

    return model


In [143]:
#Initialize keras session
tf.keras.backend.clear_session()
np.random.seed(0)
tf.random.set_seed(0)
#List parameters for different CNNs
models_config = [
  {'conv_layers': [32], 'layers': []},
  {'conv_layers' : [32, 64], 'layers' : [64, 128]},
  {'conv_layers' : [32, 64], 'layers' : [64, 128, 256]},
  {'conv_layers' : [32, 64, 128], 'layers' : [64, 64, 32]},
  {'conv_layers' : [32], 'layers' : [64, 128], 'learning_rate': 0.0005},
  {'conv_layers' : [32], 'layers' : [128, 128], 'learning_rate': 0.002},
  {'conv_layers' : [32], 'layers' : [64, 128], 'dropout': 0.6},
  {'conv_layers' : [32], 'layers' : [64, 128], 'dropout' : 0.2},
  {'conv_layers' : [32,32,32], 'layers' : [32,32,32]},
  {'conv_layers' : [32], 'dropout': 0.2, 'learning_rate': 0.0005}
]
'''
models_config = [
  {'conv_layers': [32], 'layers': []},
  {'conv_layers' : [32, 64], 'layers' : [64, 128]},
]
'''


#Create different models
models = []
for i, config in enumerate(models_config):
    # Create a CNN model with the current configuration
    model = CNNModel(class_names, **config)
    models.append(model)

In [144]:
#Average models
input_layer = tf.keras.layers.Input(shape=(480, 640, 3))  
outputs = []
for i,model in enumerate(models):
    model._name = str(i)
    output = model(input_layer)
    outputs.append(output)
concatenated_outputs = tf.keras.layers.Concatenate(axis=-1)(outputs)
output_layer = tf.keras.layers.Dense(13, activation='softmax')(concatenated_outputs)  
ensemble_model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

ResourceExhaustedError: ignored

In [ ]:
ensemble_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

history = ensemble_model.fit(
  train_ds,
  validation_data=validation_ds,
  epochs=5)

Fit ensemble model and make predictions

## Inspect model performance

130/130 - 5s - loss: 0.0276 - accuracy: 0.9916 - 5s/epoch - 37ms/step
Test loss: 0.027621300891041756 | Test accuracy: 0.9915865659713745


(0.9960734248161316, 0.9906250238418579, 0.9915865659713745)

In [ ]:
cf.EvaluateModel(model, test_ds, history)


Macro averaged precision score: 0.9915865384615383
Macro averaged recall score: 0.9916983949193181
Macro averaged F1 score: 0.9915800316396286


In [ ]:
cf.precision_recall_metrics(model, test_ds, class_names)